In [1]:
#HERE TO TEST IDEA FROM LINE CONVERSATION
import numpy as np                # import numpy
import matplotlib.pyplot as plt   # import matplotlib, a python 2d plotting library
from tqdm import tqdm
#from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM

#import torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

if torch.cuda.is_available():
  print('Running on Graphics')
  device=torch.device('cuda:0')
else:
  device=torch.device('cpu')
  print('Running on Processor')

Running on Graphics


In [8]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1 = nn.Conv2d(1,8,3, padding=1)
        self.c2 = nn.Conv2d(8,16,3, padding=1)
        self.c3 = nn.Conv2d(16,32,3, padding=1)
        self.l = nn.Linear(32,10)
        self.pool = nn.MaxPool2d(2)
        self.avgpool = nn.AvgPool2d(7)
        self.act = nn.ReLU()
        
    def forward(self,x):
        x = self.pool(self.act(self.c1(x)))
        self.feat = self.pool(self.act(self.c2(x)))
        self.maps = self.act(self.c3(self.feat))
        x = self.avgpool(self.maps).flatten(start_dim=1)
        x = self.l(x)
        return x

In [10]:
def train(optim, train_data, test_data, models, epochs, batch_size):
    metrics = []
    n = len(test_data)
    for i in tqdm(range(epochs)):
        for idx, (x, y) in enumerate(DataLoader(train_data, batch_size=batch_size, shuffle=True)):
          x = x.to(device)
          y = y.to(device)
          yhat1 = models[0](x)
          yhat2 = models[1](x)
          for model in models:
              model.zero_grad()
          loss = get_loss(y, yhat1, yhat2, models[0], models[1])
          loss.backward()
          optim.step()
        t_acc = torch.zeros(2, dtype=torch.float32)
        for idx, (x, y) in enumerate(DataLoader(test_data, batch_size=batch_size)):
          x = x.to(device)
          y = y.to(device)
          for j in range(len(models)):
              y_hat = models[j](x)
              t_acc[j] = t_acc[j] + torch.sum(torch.argmax(y_hat, dim=1)==y)
        t_acc = t_acc/n
        metrics.append(t_acc)
    return metrics

In [11]:
#Find mutual residual (proxy for mutual information)
def get_R(X,Y):
    X = torch.flatten(X, start_dim=1)
    Y = torch.flatten(Y, start_dim=1)
    #First modify to create nonsingular X:
    _,R = torch.linalg.qr(X)
    cols = torch.diag(R)
    cols = abs(cols/torch.max(cols))>0.0005
    X = X[:,cols]

    X = torch.cat([X, torch.ones([batch_size,1]).to(device)],dim=1)
    Yhat = torch.matmul(torch.matmul(X,torch.linalg.pinv(X)),Y)
    #Yhat = torch.matmul(torch.matmul(X,get_pinv(X, q)), Y)
    Ehat = Y - Yhat
    SSres = torch.sum(torch.square(Ehat))
    Ybar = torch.mean(Y, dim=0).unsqueeze(0)
    SStot = torch.sum(torch.square(Y-Ybar))
    eta = 0.001 #constant for stability
    R = 1 - SSres/(SStot+eta)
    #print('SSres:{} SStot:{} R:{}'.format(SSres, SStot, R))
    return torch.log(SStot+eta)-torch.log(SSres+eta) #R

In [22]:
lambda1 = 0.05
loss_ce = nn.CrossEntropyLoss()
def get_loss(y,yhat1,yhat2, cls1, cls2):
    L1 = loss_ce(yhat1,y)
    L2 = loss_ce(yhat2,y)
    L3 = get_R(cls1.feat, cls2.feat)
    #print('L1:{} L2:{} L3:{}'.format(L1,L2,L3))
    return L1+L2+lambda1*L3

In [23]:
def get2classfiers(epochs):
    cls1 = Classifier().to(device)
    cls2 = Classifier().to(device)
    optimizer = optim.Adam(list(cls1.parameters())+list(cls2.parameters()), lr = 5.0e-3)
    loss_ce = nn.CrossEntropyLoss()
    models = [cls1, cls2]
    metric = train(optimizer, train_data, test_data, models, epochs, batch_size)
    print('Model 1 Accuracy:{}%   Model 2 Accuracy:{}%'.format(metric[-1][0],metric[-1][1]))
    return cls1, cls2

In [24]:
def gen_FGSM(x, y, eta, model):
    model.zero_grad()
    x.requires_grad = True
    y_hat = model(x)
    loss = loss_ce(y_hat, y)
    loss.backward()
    perturbed_x = torch.clamp(x + eta*(x.grad.data).sign(), min=0, max=1.0)
    return perturbed_x#, x.grad.data

In [25]:
def get_transfer(etas):
    trans_rate = np.zeros(len(etas))
    R2 = np.zeros(len(etas))
    cls1, cls2 = get2classfiers(epochs)
    for eidx, eta in enumerate(etas):
        for x,y in DataLoader(test_data, batch_size, shuffle=True):
            x = x.to(device)
            y = y.to(device)
            x = gen_FGSM(x,y,eta,cls1)
            c1=(torch.argmax(cls1(x), dim=1)==y)
            c2=(torch.argmax(cls2(x), dim=1)==y)
            adv_tran = torch.sum(~c1 & ~c2)/torch.sum(~c1)
            trans_rate[eidx] = trans_rate[eidx] + adv_tran#.detach().cpu().numpy()
            R2[eidx] = R2[eidx] + get_R(cls1.feat, cls2.feat)
    trans_rate = trans_rate*batch_size/(len(test_data))
    R2 = R2*batch_size/(len(test_data))
    
    return trans_rate, R2, cls1, cls2
                

In [9]:
#Load Data
train_data = MNIST('../mnist_digits/', train=True, download=True,transform=torchvision.transforms.ToTensor())
test_data = MNIST('../mnist_digits/', train=False, download=True,transform=torchvision.transforms.ToTensor())

Processing...
Done!


/home/wiedec/anaconda3/envs/torch/lib/python3.7/site-packages/torchvision/datasets/mnist.py:299: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554827596/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed).view(length, num_rows, num_cols)


In [26]:
etas = [0.0, 0.05, 0.1, 0.15]
epochs=20
batch_size=1000
trans_rate, R2, cls1, cls2 = get_transfer(etas)

100%|██████████| 20/20 [01:45<00:00,  5.26s/it]


Model 1 Accuracy:0.9375%   Model 2 Accuracy:0.9609000086784363%


In [28]:
trans_rate

array([0.3934622 , 0.32310801, 0.52864656, 0.70998712])

In [ ]:
np.savez('transfer_rate_decorrelated_cnns', trans_rate, R2_diff)